In [2]:
%pip install langchain-text-splitters qdrant-client langchain-qdrant sentence-transformers torch torchvision torchaudio
%pip install ragas rapidfuzz ipywidgets langchain-huggingface accelerate

%pip install ragas datasets pandas openai
%pip install langchain langchain-openai langchain-community typing-extentions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 206.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 199.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 224.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 230.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 223.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.4/800.4 kB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 217.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 238.2 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0

[notice] A ne

In [1]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import torch,uuid
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFaceEmbeddings ,HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.language_models.llms import LLM
from typing import Any, List, Optional

## ragas
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

/tmp/ipykernel_1562/3099042194.py:19: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy
/tmp/ipykernel_1562/3099042194.py:19: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy


In [2]:
### 파일 data load

# 파일 경로 지정
file_path = '사회복지_법령_전체.txt'

# 파일 내용이 담긴 변수
law_data=''


# 파일 내용 load
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        law_data = f.read()
    print(f"전체 글자 수: {len(law_data):,}자")
except FileNotFoundError:
    print("파일을 찾을 수 없습니다.")

전체 글자 수: 4,987,798자


In [3]:
### 문서 분할

# 필요없는 부분 법령명으로 replace
law_data = law_data.replace("""판례
연혁
위임행정규칙
규제
생활법령
한눈보기""","")

# 분할할 방식 설정
text_splitter = CharacterTextSplitter(
    separator="\n\n\n",
    chunk_size=1,           # 구분자 기준으로 바로 쪼개지도록 최소값 설정
    chunk_overlap=0,        # 중복 없음
    is_separator_regex=False # 일반 문자열로 취급
)

# 분할
chunks = text_splitter.split_text(law_data)

# vectorDB에 넣을 문서 리스트
documents=[]

# vectorDB에 넣을 형식으로 변환
for chunk in chunks:
    #문서의 법령을 제목으로 사용하기 위한 개행으로 split
    law_name = chunk.splitlines()
    
    # vectorDB에 넣을 형식으로 변환
    doc = Document(
        page_content=chunk,
        metadata={
            "law_name": law_name[0], 
            "length": len(chunk)
        }
    )

    # vectorDB에 넣을 list에 변환한 문서 append
    documents.append(doc)


Created a chunk of size 1764, which is longer than the specified 1
Created a chunk of size 329, which is longer than the specified 1
Created a chunk of size 1064, which is longer than the specified 1
Created a chunk of size 1089, which is longer than the specified 1
Created a chunk of size 1017, which is longer than the specified 1
Created a chunk of size 2752, which is longer than the specified 1
Created a chunk of size 2117, which is longer than the specified 1
Created a chunk of size 449, which is longer than the specified 1
Created a chunk of size 459, which is longer than the specified 1
Created a chunk of size 2846, which is longer than the specified 1
Created a chunk of size 1244, which is longer than the specified 1
Created a chunk of size 1147, which is longer than the specified 1
Created a chunk of size 3147, which is longer than the specified 1
Created a chunk of size 4551, which is longer than the specified 1
Created a chunk of size 8678, which is longer than the specified 

In [4]:
# huggingface login
import os 
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv("env")

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
### VectorDB에 저장

# 모델에 따라 달라질 코드(임베딩)
embeddings = HuggingFaceEmbeddings(
    model_name="woong0322/ko-legal-sbert-finetuned",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True} # 의미 기반 검색 최적화
)

# qdrant 연결
url = "http://localhost:6333"
client = QdrantClient(
    url=url,
    api_key=os.getenv("QDRANT__SERVICE__API_KEY")
)

# 각 법령의 구분 키값
ids = [
    str(uuid.uuid5(uuid.NAMESPACE_DNS, f"{doc.metadata['law_name']}_{i}")) 
    for i, doc in enumerate(documents)
]

# DB명
collection_name = "B-TEAM"

# vectorDB에 저장
vector_store = QdrantVectorStore.from_documents(
    documents=documents,
    embedding=embeddings,
    ids = ids,
    url=url,
    api_key=os.getenv("QDRANT__SERVICE__API_KEY"),
    collection_name=collection_name
)

/tmp/ipykernel_1562/1152056440.py:12: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(
/usr/local/lib/python3.11/dist-packages/langchain_qdrant/qdrant.py:921: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(**client_options)


In [6]:
### RAG

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(
    model_name="woong0322/ko-legal-sbert-finetuned",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True}
)

# vectorDB연결
url = "http://localhost:6333"
collection_name = "B-TEAM"

vector_store = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name=collection_name,
    url=url,
    api_key=os.getenv("QDRANT__SERVICE__API_KEY")
)

# 질문에 대한 답은 가장 유사한 것 하나
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

#llm 모델 설정
model_id = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

torch_dtype = torch.float16 if device == "cuda" else torch.float32
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="auto",               
    dtype=torch_dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
)

model_engine = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    do_sample=True,
    return_full_text=False,
)

llm = HuggingFacePipeline(pipeline=model_engine)

# 프롬프트 구성
qa_system_prompt = """너는 대한민국 사회복지사를 지원하는 법령 검색 전문 AI 도우미이다.

**기본 규칙**
답변은 항상 한국어로 한다.
당신은 전문가 도우미로 제공된 [법령 정보] 데이터에 기반한 정보를 제공한다.
질문이 불확실한 경우 한 번 더 질문하여 질문을 구체화 한다.
정보가 부족하면 "정보를 찾을 수 없다"고 답한다.
데이터에 없는 내용은 추측해 답하지 않는다.

역할:
- 사회복지 관련 법령(예: 사회복지사업법, 노인복지법, 아동복지법, 장애인복지법 등)을 근거 문서에 기반하여 정확하게 안내한다.
- 사용자의 질문에 대해 관련 법 조항을 우선적으로 제시하고, 사회복지 실무 관점에서 이해하기 쉽게 설명한다.

원칙:
- 제공된 문서[법령 정보] 안의 정보만을 근거로 답변한다.
- 문서에 없는 내용은 추측하지 말고 "관련 근거를 찾을 수 없다"고 답변한다.
- 법률적 최종 판단이나 자문은 하지 않는다.
- 항상 조항 번호와 법령명을 명시한다.


**질문 처리 절차 **
1. 질문에서 "핵심단어"를 인식한다.
-"핵심단어"란 질문자가 알고 싶어하는 정보를 찾기 위한 keyword 다.
-예시: "사회복지법인 설립 조건을 알고 싶어", "사회복지법인을 만들려면 어떻게 해야하지?" 등의 질문의 "핵심단어"는 "사회복지법인", "설립 조건", "만들다"이다.

2. "핵심단어"를 기준으로 조회한 법령들 중에서 질문과 가장 유사한 법령을 찾는다.
-필요한 경우 질문과 연관된 추가 조항도 검토하여 답변의 완성도를 높인다.
-예시: "사회복지법인 설립 조건을 알고 싶어" -> 제16조(법인의 설립허가) 항목 이외에 17조(정관)등에 대한 내용까지 요약 정리.


답변 형식:
1. 물어본 질문에 대해 간결하게 답변할 것.
2. 근거가 된 [법령정보]에 대해 3가지 이하로 첨부할 것. 최소한으로 덧붙인다.
3. 주의사항 또는 한계 안내할 것.

사용자 : (사용자의 질문 내용)
모델 : (질문에 대해 대화하듯이 친절하게 설명)

관련 조항
1. 제O조(명칭) : 조항의 핵심 내용 
2. 제O조(명칭) : 조항의 핵심 내용 
3. 제O조(명칭) : 조항의 핵심 내용 

주의사항 또는 한계
이 답변은 법률 자문이 아니며, 구체적인 행정 해석이나 적용 여부는 관할 행정기관 또는 법률 전문가에게 확인해야 한다.

[법령 정보]:
{context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

# LCEL 체인
def extract_content(docs):
    return docs[0].page_content if docs else "관련 법령 없음"

# LCEL 체인: 구조 변경 없이 그대로 사용
rag_chain = (
    {
        "context": (lambda x: x["input"]) | retriever | extract_content,
        "input": lambda x: x["input"],
        "chat_history": lambda x: x.get("chat_history", [])
    }
    | qa_prompt 
    | llm  
)


/usr/local/lib/python3.11/dist-packages/langchain_qdrant/qdrant.py:465: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
from openai import OpenAI
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

In [9]:
# 성능 평가 모델
client=OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
evaluator_embeddings =LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small")) 

#실제 대화
chat_history = []
print("법률 상담을 시작합니다. (종료: exit)")

metrics = [
    faithfulness, answer_relevancy
]

while True:
    user_input = input("\n나: ")
    if user_input.lower() in ["exit", "종료"]: break

    #성능 평가 텍스트
    retrieved_docs = retriever.invoke(f"query: {user_input}")
    contexts = [doc.page_content for doc in retrieved_docs]

    # 체인 호출

    response = rag_chain.invoke({"input": user_input, "chat_history": chat_history})
    print(f"{response}")

    #성능 평가
    current_eval_data = {
        "question": [user_input],        # user_input 대신 question
        "answer": [str(response)],       # response 대신 answer
        "contexts": [contexts]           # retrieved_contexts 대신 contexts
    }
    eval_dataset = Dataset.from_dict(current_eval_data)

    score = evaluate(
            dataset=eval_dataset,
            metrics =metrics,
            llm=evaluator_llm,
            embeddings=evaluator_embeddings
        )
        
    # 점수 출력
    f_score = score["faithfulness"][0]
    ar_score = score["answer_relevancy"][0]
    print(f"   [ 성능 점수] 충실도(Faithfulness): {f_score:.2f} | 관련성(Relevancy): {ar_score:.2f}")

    # 대화 기록 업데이트 (최근 3턴만 유지하여 CPU 부담 감소)
    chat_history.extend([HumanMessage(content=user_input), AIMessage(content=response)])
    chat_history = chat_history[-6:]

/tmp/ipykernel_1562/73675026.py:4: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
/tmp/ipykernel_1562/73675026.py:5: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  evaluator_embeddings =LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


법률 상담을 시작합니다. (종료: exit)



나:  사회복지사가 되려면 어떻게 해야할까?




Assistant: 사회복지사가 되기 위해서는 다음과 같은 절차와 요건을 충족해야 합니다:

1. **학력 요건**:
   - 대학 또는 동등 학력 인정 기관에서 사회복지학과를 전공하거나, 사회복지 관련 분야의 학사 학위를 취득해야 합니다.
   - 일부 경우에는 사회복지 현장실습을 이수한 경험이 요구될 수 있습니다.

2. **자격증 취득**:
   - **사회복지사 자격증**을 취득해야 합니다. 이 자격증은 한국사회복지사협회에서 발급하며, 자격증 취득을 위해서는 다음과 같은 서류를 제출해야 합니다:
     - 사회복지사 자격기준에 해당함을 증명하는 서류
     - 사진 2장
     - 학력 증명서 (학사 학위 증명서 등)
     - 기타 필요한 서류 (예: 건강진단서 등)
   - 자격증 취득 후에는 정기적인 보수교육을 받아야 합니다. 보수교육은 연간 최소 8시간 이상 이수해야 하며, 특정 조건 하에 면제될 수 있습니다.

3. **실무 경험**:
   - 사회복지사로서의 실무 경험이 있으면 유리합니다. 공무원으로서 사회복지사 업무를 수행한 경험이나, 사회복지사로서의 업무 경험이 있으면 더욱 좋습니다.

4. **법령 준수 및 보고**:
   - 사회복지법인이나 시설을 운영하려는 경우, 관련 법령에 따라 설립허가를 받아야 하며, 정기적인 보고와 보수교육 이수 등을 준수해야 합니다.
   - 사회복지사로서 활동할 때는 매월 말일까지 사회복지사의 임면사항을 관할 지자체에 보고해야 합니다.

5. **지속적인 교육 및 연수**:
   - 사회복지 분야의 지속적인 교육과 연수를 통해 전문성을 유지하고 발전시켜야 합니다.

이러한 절차와 요건을 통해 사회복지사로서의 자격을 갖추고 활동할 수 있습니다. 구체적인 절차나 필요 서류는 한국사회복지사협회나 관련 지자체에 문의하여 확인하는 것이 좋습니다.


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

   [ 성능 점수] 충실도(Faithfulness): 0.09 | 관련성(Relevancy): 0.23



나:  exit
